In [1]:
import pandas as pd 

C:\Users\PC\AppData\Local\Temp\ipykernel_21912\217833862.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
df = pd.read_csv(r'D:\Proyectos\nllp_antitrust.v1\corpus.csv')
df = df.dropna()
df = df.applymap(lambda x: x.replace('\n', ' ') if isinstance(x, str) else x)
df = df.drop(columns=['Page', 'Text','nature_of_case','fact_summary'])
df.reset_index(drop=True,inplace=True)

C:\Users\PC\AppData\Local\Temp\ipykernel_21912\2133272646.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.replace('\n', ' ') if isinstance(x, str) else x)


In [4]:
df

,case_name,rule_of_law
0,United States v. Trans-Missouri Freight Assn. ...,The Sherman Act’s prohibition on agreements in...
1,United States v. Addyston Pipe & Steel Co. Fed...,A combination with the sole purpose to regulat...
2,"Summit Health, Ltd. v. Pinhas Health care oper...",Violation of the Sherman Act is determined by ...
3,Illinois Brick Co. v. Illinois Manufacturer (D...,Only direct purchasers from those engaged in u...
4,Reiter v. Sonotone Corp. Consumer (P) v. Manuf...,A consumer of retail goods and services has st...
...,...,...
104,Fisher v. City of Berkeley Landlord (P) v. Cit...,Rent control ordinances do not conflict with f...
105,California Retail Liquor Dealers Assn. v. Midc...,A state policy is immune from antitrust regula...
106,Hallie v. City of Eau Claire Unincorporated to...,A municipality’s anticompetitive activities ar...
107,City of Columbia & Columbia Outdoor Advertisin...,"A city may, through regulation, give a busines..."


In [5]:
import nltk
nltk.download("punkt")
nltk.download("stopwords")
nltk.download("wordnet")
from nltk import word_tokenize, FreqDist
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [6]:
stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

def text_preprocess(text: str) -> str:
    tokens = [lemmatizer.lemmatize(token) for token in word_tokenize(text.lower()) if (token.isalpha() and (token not in stop_words or token == "per" or token == "se"))]

    return ' '.join(tokens)

In [7]:
df["rol_preprocessed"] = df.rule_of_law.apply(text_preprocess)

In [10]:
df

,case_name,rule_of_law,rol_preprocessed
0,United States v. Trans-Missouri Freight Assn. ...,The Sherman Act’s prohibition on agreements in...,sherman act prohibition agreement restraint tr...
1,United States v. Addyston Pipe & Steel Co. Fed...,A combination with the sole purpose to regulat...,combination sole purpose regulate price violat...
2,"Summit Health, Ltd. v. Pinhas Health care oper...",Violation of the Sherman Act is determined by ...,violation sherman act determined potential har...
3,Illinois Brick Co. v. Illinois Manufacturer (D...,Only direct purchasers from those engaged in u...,direct purchaser engaged unlawful manufacturin...
4,Reiter v. Sonotone Corp. Consumer (P) v. Manuf...,A consumer of retail goods and services has st...,consumer retail good service standing sue dama...
...,...,...,...
104,Fisher v. City of Berkeley Landlord (P) v. Cit...,Rent control ordinances do not conflict with f...,rent control ordinance conflict federal antitr...
105,California Retail Liquor Dealers Assn. v. Midc...,A state policy is immune from antitrust regula...,state policy immune antitrust regulation clear...
106,Hallie v. City of Eau Claire Unincorporated to...,A municipality’s anticompetitive activities ar...,municipality anticompetitive activity protecte...
107,City of Columbia & Columbia Outdoor Advertisin...,"A city may, through regulation, give a busines...",city may regulation give business concern de f...


In [9]:
df.to_csv(r'D:\Proyectos\nllp_antitrust.v1\corpus_rol.csv', index=False)

3. We will create de document term matrix

In [12]:
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer(min_df=0.01, # at min 1% of docs
                        max_df=.9,  
                        max_features=20,
                        ngram_range=(1,2))
X = vec.fit_transform(df['rol_preprocessed'])

# save the vectors
pd.to_pickle(X,r'D:\Proyectos\nllp_antitrust.v1\X.pkl')

# Do more research in this regard 
